In [1]:
import torch
from transformers import AutoTokenizer, AutoModelFobrCausalLM
from datasets import load_dataset

ModuleNotFoundError: No module named 'torch'

In [ ]:
model_path = "../outputs/qwen3-0.6B-gsm8k-full"

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad token is set

gsm8k = load_dataset("gsm8k", "main", split="test")


In [ ]:
def format_prompt(question):
    return f"### Question:\n{question.strip()}\n\n### Answer:\n"

def get_model_answer(prompt, model, tokenizer, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.7,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
for i in range(5):  # Try first 5 examples
    question = gsm8k[i]["question"]
    correct_answer = gsm8k[i]["answer"].split("####")[-1].strip()
    prompt = format_prompt(question)
    response = get_model_answer(prompt, model, tokenizer)
    print(f"Example {i+1}")
    print("Question:", question)
    print("Model response:", response)
    print("Correct answer:", correct_answer)
    print("-" * 60)